## Imports and settings

In [6]:
import os
import pandas as pd

path_data_raw = '../data/raw/'
fname_data_raw = 'rse_group_survey_2023_responses'

In [16]:
dset = pd.read_csv(f"{path_data_raw}{fname_data_raw}.csv")
qtxt = dset.columns[1:].astype(str).tolist()

print(f"Questions  : {len(qtxt)}")
print(f"Respondents: {dset.shape[0]}")

Questions  : 20
Respondents: 36
